You have been hired as a data engineer by research organization. Your boss has asked you to create a code that can be used to compile the list of the top 10 largest banks in the world ranked by market capitalization in billion USD. Further, the data needs to be transformed and stored in GBP, EUR and INR as well, in accordance with the exchange rate information that has been made available to you as a CSV file. The processed information table is to be saved locally in a CSV format and as a database table.

Your job is to create an automated system to generate this information so that the same can be executed in every financial quarter to prepare the report.

In [1]:
!pip install numpy pandas bs4 wget


  Preparing metadata (setup.py) ... done
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9656 sha256=e5410475f0f00690c3978bfe46dd75c66bc8dff73fc8c08dcad331e6516cadbc
  Stored in directory: /root/.cache/pip/wheels/40/b3/0f/a40dbd1c6861731779f62cc4babcb234387e11d697df70ee97
Successfully built wget


In [1]:
import pandas as pd
import numpy as np
import sqlite3
import requests
from datetime import datetime
from bs4 import BeautifulSoup

Logging Process

In [4]:
def log_progress(message):
  timestamp_format="%y-%h_d-%H:%M:%S"
  now=datetime.now()
  timestamp=now.strftime(timestamp_format)
  with open(log_file,"a") as f:
    f.write(timestamp+":"+message +"\n")

In [6]:
def extract(url, table_attribs):
    ''' This function aims to extract the required
    information from the website and save it to a data frame. The
    function returns the data frame for further processing. '''

    page = requests.get(url).text
    soup = BeautifulSoup(page, "html.parser")

    df = pd.DataFrame(columns=table_attribs)

    tables = soup.find_all("tbody")
    rows = tables[0].find_all("tr")

    for row in rows:
        col = row.find_all("td")
        if len(col) != 0:
            data_dict = {"Name": col[1].find_all("a")[1]["title"],
                         "MC_USD_Billion": float(col[2].contents[0][:-1])}
            df1 = pd.DataFrame(data_dict, index=[0])
            df = pd.concat([df, df1], ignore_index=True)

    return df


In [7]:
def transform(df, csv_path):
    ''' This function accesses the CSV file for exchange rate
    information, and adds three columns to the data frame, each
    containing the transformed version of Market Cap column to
    respective currencies'''

    # Read exchange rate CSV file
    exchange_rate = pd.read_csv(csv_path)

    # Convert to a dictionary with "Currency" as keys and "Rate" as values
    exchange_rate = exchange_rate.set_index("Currency").to_dict()["Rate"]

    # Add MC_GBP_Billion, MC_EUR_Billion, and MC_INR_Billion
    # columns to dataframe. Round off to two decimals
    df["MC_GBP_Billion"] = [np.round(x * exchange_rate["GBP"], 2) for x in df["MC_USD_Billion"]]
    df["MC_EUR_Billion"] = [np.round(x * exchange_rate["EUR"], 2) for x in df["MC_USD_Billion"]]
    df["MC_INR_Billion"] = [np.round(x * exchange_rate["INR"], 2) for x in df["MC_USD_Billion"]]

    return df

In [8]:
def load_to_csv(df, output_path):
    ''' This function saves the final data frame as a CSV file in
    the provided path. Function returns nothing.'''

    df.to_csv(output_path)

Loading to Database

In [9]:
def load_to_db(df, sql_connection, table_name):
    ''' This function saves the final data frame to a database
    table with the provided name. Function returns nothing.'''

    df.to_sql(table_name, sql_connection, if_exists='replace', index=False)

Function to run querries on database

In [10]:
def run_query(query_statement, sql_connection):
    ''' This function runs the query on the database table and
    prints the output on the terminal. Function returns nothing. '''

    print(query_statement)
    query_output = pd.read_sql(query_statement, sql_connection)
    print(query_output)

Running ETl Process

In [11]:
url = "https://web.archive.org/web/20230908091635/https://en.wikipedia.org/wiki/List_of_largest_banks"
csv_path = "./exchange_rate.csv"
table_attribs = ["Name", "MC_USD_Billion"]
output_path = "./Largest_banks_data.csv"
db_name = "Banks.db"
table_name = "Largest_banks"
log_file = "./code_log.txt"

log_progress("Preliminaries complete. Initiating ETL process")

calling extract function

In [12]:
df = extract(url, table_attribs)
print(df)

log_progress("Data extraction complete. Initiating Transformation process")

                                      Name  MC_USD_Billion
0                           JPMorgan Chase          432.92
1                          Bank of America          231.52
2  Industrial and Commercial Bank of China          194.56
3               Agricultural Bank of China          160.68
4                                HDFC Bank          157.91
5                              Wells Fargo          155.87
6                                     HSBC          148.90
7                           Morgan Stanley          140.83
8                  China Construction Bank          139.82
9                            Bank of China          136.81


<ipython-input-6-9395c0be0ee4>:20: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df, df1], ignore_index=True)


In [13]:
df = transform(df, csv_path)
print(df)

log_progress("Data transformation complete. Initiating Loading process")


                                      Name  MC_USD_Billion  MC_GBP_Billion  \
0                           JPMorgan Chase          432.92          346.34   
1                          Bank of America          231.52          185.22   
2  Industrial and Commercial Bank of China          194.56          155.65   
3               Agricultural Bank of China          160.68          128.54   
4                                HDFC Bank          157.91          126.33   
5                              Wells Fargo          155.87          124.70   
6                                     HSBC          148.90          119.12   
7                           Morgan Stanley          140.83          112.66   
8                  China Construction Bank          139.82          111.86   
9                            Bank of China          136.81          109.45   

   MC_EUR_Billion  MC_INR_Billion  
0          402.62        35910.71  
1          215.31        19204.58  
2          180.94        16138.75

calling load function

In [14]:
load_to_csv(df, output_path)
log_progress("Data saved to CSV file")

Initiate SQLite3 connection



In [15]:
sql_connection = sqlite3.connect(db_name)

log_progress("SQL Connection initiated")


Load to DataBase

In [16]:
load_to_db(df, sql_connection, table_name)

log_progress("Data loaded to Database as a table, Executing queries")

Run querry


1.  print the content of the entre table




In [17]:
query_statement = f"SELECT * from {table_name}"
run_query(query_statement, sql_connection)

SELECT * from Largest_banks
                                      Name  MC_USD_Billion  MC_GBP_Billion  \
0                           JPMorgan Chase          432.92          346.34   
1                          Bank of America          231.52          185.22   
2  Industrial and Commercial Bank of China          194.56          155.65   
3               Agricultural Bank of China          160.68          128.54   
4                                HDFC Bank          157.91          126.33   
5                              Wells Fargo          155.87          124.70   
6                                     HSBC          148.90          119.12   
7                           Morgan Stanley          140.83          112.66   
8                  China Construction Bank          139.82          111.86   
9                            Bank of China          136.81          109.45   

   MC_EUR_Billion  MC_INR_Billion  
0          402.62        35910.71  
1          215.31        19204.58  
2    

Printing average market capitalization of all  banks in billion GBP

In [18]:
query_statement = f"SELECT AVG(MC_GBP_Billion) FROM {table_name}"
run_query(query_statement, sql_connection)

SELECT AVG(MC_GBP_Billion) FROM Largest_banks
   AVG(MC_GBP_Billion)
0              151.987


Print the name of the top bank

In [19]:
query_statement = f"SELECT Name from {table_name} LIMIT 5"
run_query(query_statement, sql_connection)

log_progress("Process Complete")

SELECT Name from Largest_banks LIMIT 5
                                      Name
0                           JPMorgan Chase
1                          Bank of America
2  Industrial and Commercial Bank of China
3               Agricultural Bank of China
4                                HDFC Bank


SQL 3 connection close

In [20]:
sql_connection.close()

log_progress("Server Connection closed")

Verify Log entries
upon successful completion of executation ,see the relevent entries made in the log file in relation to the stages of code executation.

In [21]:
with open(log_file, "r") as log:
    LogContent = log.read()
    print(LogContent)


25-Mar_d-08:43:02:Preliminaries complete. Initiating ETL process
25-Mar_d-08:43:07:Data extraction complete. Initiating Transformation process
25-Mar_d-08:43:10:Data transformation complete. Initiating Loading process
25-Mar_d-08:43:13:Data saved to CSV file
25-Mar_d-08:43:14:SQL Connection initiated
25-Mar_d-08:43:16:Data loaded to Database as a table, Executing queries
25-Mar_d-08:43:22:Process Complete
25-Mar_d-08:43:23:Server Connection closed

